In [ ]:
import keysight_kt34400 as m
import numpy as np # Required by keysight_kt34400
import time
import datetime
import os

In [ ]:
def main():
    """
    main() class for keysight_kt34400 driver. mostly copy pasted from the "help" folder, "example code" section,
    found inside the drivers for the kt34400. Currently using a kt34461a, all commands seem compatible

    'resource_name' is a VISA-COM alias address that I obtained by turning the multimeter on, then going 
    to Utility -> I/O Config -> UsB Settings -> Show USB ID
    """
    resource_name = "USB0::10893::5121::SG57200293::0::INSTR"
    idQuery = True
    reset   = True
    options = "QueryInstrStatus=False, Simulate=False, Trace=False"
    
    
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    #--------------------DMM Driver Initilization---------------------
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    try:
        # Call driver constructor with options
        global driver # May be used in other functions
        driver = None
        driver = m.Kt34400(resource_name, idQuery, reset, options)
        print("Driver Initialized")

        #  Print a few identity properties
        print('  identifier: ', driver.identity.identifier)
        print('  revision:   ', driver.identity.revision)
        print('  vendor:     ', driver.identity.vendor)
        print('  description:', driver.identity.description)
        print('  model:      ', driver.identity.instrument_model)
        print('  resource:   ', driver.driver_operation.io_resource_descriptor)
        print('  options:    ', driver.driver_operation.driver_setup)
       
    # Check instrument error queue for errors, print every single one
        while True:
            outVal = ()
            outVal = driver.utility.error_query()
            print("  error_query: code:", outVal[0], " message:", outVal[1])
            print("\n")
            if(outVal[0] == 0): # 0 = No error, error queue empty
                break    
                
        # Reset the DMM
        driver.utility.reset()

        
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    #-------------------File & Directory Creation---------------------
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
        # locate desktop and designate a folder for storing data
        desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') 
        script_path = os.path.join(desktop, "ProSense_Flowmeter_Script\\")
        data_path = os.path.join(script_path, "Data\\")
        
        # create directory if script folder is missing
        if not os.path.exists(script_path):
            os.makedirs(script_path)
            
        # create directory if data folder is missing   
        if not os.path.exists(data_path):
            os.makedirs(data_path)
                              
        # create text file with today's date as name
        today = datetime.date.today()
        filename = today.strftime("%m_%d_%y") + ".txt"   # e.g. 09_13_22.txt
        try:
            with open(os.path.join(data_path, filename), 'a') as f:  # use append mode in case file exists already
                date = str(datetime.datetime.now().strftime("%D %I:%M:%S %p"))  #Mon-Day-Yr  Hr:Min:Sec am/pm
                f.write('Initializing flowrate data collection: ' + date + '\n')
                
        except FileNotFoundError:
            print("except: FileNotFoundError")
    
    
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    #-------------------DC Current setup------------------------------
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
        # Set up the DMM for immediate trigger
        #driver.trigger.source = m.TriggerSource.IMMEDIATE
        driver.dc_current.configure()
        
        max_time = datetime.timedelta(seconds = 5)
        
        # test function
        '''
        for num in range(5):
            time.sleep(1)
            data = driver.measurement.read(max_time)
            print("DC Current measurement: ", data, " Amps")
        '''
        
        
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    #----------------DC Current Measurement & Saving------------------
    #"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    # every 30s, ping multimeter, print to console, and append data line to file
        while True: 
            # first, sleep for 30s
            time.sleep(30)
            data = driver.measurement.read(max_time)  # round to 5 digits to match DMM display
            data *= 1e3  # convert 0.004 V measurement to 4 mV
            
            # convert mV measurement to flowrate
            ### range is from 4mV to 20mV gpm
            ### flow is from ASP to AEP, by default 0.00 to 5.28 gpm
            
            SSP = 4     # signal start point [mV]
            SEP = 20    # signal end point [mV]
            ASP = 0.00  # analog start point [gpm]
            AEP = 5.28  # analog end point [gpm]
           
            # shift data down by 4mV (SSP), then 
            # divide by 16mV (SEP-SSP) to see the percentage. 
            # then convert this to flowrate using the meter range
            percentage = (data-SSP)/(SEP-SSP)
            flow_rate = percentage*(AEP-ASP)
            
            # this will be appended to the text file
            
            # now save data to opened file
            try:
                with open(os.path.join(data_path, filename), 'a') as f:  # use append mode to... append to today's file
                    timestamp = str(datetime.datetime.now().strftime("%I:%M:%S %p")) # Hr:Min:Sec am/pm
                    data_measurement_string = "[{0}] {1:1.2f} GPM @ {2:1.5f} mV  \n".format(timestamp, flow_rate, data)

                    print(data_measurement_string)  # record in console
                    f.write(data_measurement_string)  # record in text file
                    
            except FileNotFoundError:
                print("except: FileNotFoundError")


    except Exception as e:
        print("\n  Exception:", e.__class__.__name__, e.args)

    finally:
        if driver is not None: # Skip close() if constructor failed
            driver.close()
        #input("\nDone - Press Enter to Exit")

        
        
if __name__ == "__main__":
    main()
    
   